In [49]:
import wave
import os
import struct
# import sounddevice as sd
from scipy.io.wavfile import write
import soundfile as sf
# import simpleaudio
import pyaudio
# import speech_recognition as sr
import requests
from sklearn.metrics import mean_squared_error
import librosa
import pandas as pd
from librosa import display

import matplotlib.pyplot as plt

import numpy as np
from scipy.fftpack import fft

In [50]:
def soundFFT(filename):
  samples, sampling_rate = librosa.load(filename, sr = None, mono = True, offset = 0, duration = None)
  print(len(samples), sampling_rate, len(samples)/sampling_rate)
  n = (len(samples))
  T = 1/sampling_rate
  yf = fft(samples)
  xf = np.linspace(0.0, 1.0/(2.0*T), (int)(n/2))
  # fig, ax = plt.subplots()
  yplot = 2.0/n*np.abs(yf[:n//2])
  cutoff = 0
  for i in range(len(xf)):
    if xf[i] > 4000:
      cutoff = i
      break
  if cutoff > 0:
    # ax.plot(xf[:cutoff], yplot[:cutoff])
    filtered_freq = pd.DataFrame({'magnitude': yplot, 'freq':xf})
  else:
    # ax.plot(xf, yplot)
    filtered_freq = pd.DataFrame({'magnitude': yplot, 'freq':xf})
  
  # print(yf[])
  print(yf[(n//2)-3:(n//2) +3])
  # plt.xlabel("Frequency -->")
  # plt.ylabel("Magnitude")
  return filtered_freq

soundFFT('stream.wav')

C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\722879639.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, sampling_rate = librosa.load(filename, sr = None, mono = True, offset = 0, duration = None)
c:\Users\S-_-z\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'stream.wav'

In [ ]:
def recordAudio(filename):
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 1
    fs = 44100  # Record at 44100 samples per second
    seconds = 10

    p = pyaudio.PyAudio()  # Create an interface to PortAudio
    print('Recording')

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []  # Initialize array to store frames

    # Store data in chunks for 3 seconds
    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()

    print('Finished recording')

    # Save the recorded data as a WAV file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()


In [52]:
def initAlarmCSV():
  file = 'alarm_mono_single_pi_10.wav'
  filtered1 = soundFFT(file)

  freqs = np.arange(0, 4000, 1)
  merged_df = pd.DataFrame({'freq': freqs})
  merged_df['freq1'] = pd.Series()
  merged_df['mag1'] = pd.Series()

  #filetered1
  n = 0
  indexFreq = 0
  for freq in freqs:
    tempIndex = n
    while abs((freq - filtered1['freq'][tempIndex])) > abs((freq - filtered1['freq'][tempIndex+1])):
      tempIndex += 1
    
    merged_df['freq1'][indexFreq] = filtered1['freq'][tempIndex]
    merged_df['mag1'][indexFreq] = filtered1['magnitude'][tempIndex]

    n = tempIndex
    n+=1
    indexFreq+=1

  merged_df.to_csv('alarm.csv', index=False)

initAlarmCSV()

440320 44100 9.984580498866213
[-0.0851469 -0.02552339j  0.06445333-0.01903605j -0.03401828+0.05426836j
  0.10812378-0.j         -0.03401828-0.05426836j  0.06445333+0.01903605j]


C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\4244933189.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['freq1'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\4244933189.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['mag1'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\4244933189.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['freq1'][indexFreq] = filtered1['freq'][tempIndex]
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\4244933189.py:19: SettingWithCopyWarning: 
A value is trying to

In [53]:
def freqMagMSE(filtered2):
  # Matches up different size and index dataframes in one merged df
  # Takes input two dataframes with columns: 'freq' and 'magnitude
  # Match the closest frequencies up to 4000, then add corresponding magnitudes
  freqs = np.arange(0, 4000, 1)
  merged_df = pd.read_csv('alarm.csv')
  merged_df['freq2'] = pd.Series()
  merged_df['mag2'] = pd.Series()

  #filtered2
  n = 0
  indexFreq = 0
  for freq in freqs:
    tempIndex = n
    while abs((freq - filtered2['freq'][tempIndex])) > abs((freq - filtered2['freq'][tempIndex+1])):
      tempIndex += 1
    
    merged_df['freq2'][indexFreq] = filtered2['freq'][tempIndex]
    merged_df['mag2'][indexFreq] = filtered2['magnitude'][tempIndex]

    n = tempIndex
    n+=1
    indexFreq+=1
      
  return mean_squared_error(merged_df['mag1'], merged_df['mag2'])

file = 'test.wav'
recordAudio(file)
filtered = soundFFT(file)
freqMagMSE(filtered)

Recording
Finished recording
440320 44100 9.984580498866213
[-0.00064342-0.01302845j -0.00225024-0.00105667j -0.00179355+0.00857719j
  0.08111572-0.j         -0.00179355-0.00857719j -0.00225024+0.00105667j]


C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['freq2'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['mag2'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['freq2'][indexFreq] = filtered2['freq'][tempIndex]
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:19: SettingWithCopyWarning: 
A value is trying to be 

1.019886678434893e-09

In [54]:
file = 'test2.wav'
recordAudio(file)
filtered = soundFFT(file)
freqMagMSE(filtered)

Recording
Finished recording
440320 44100 9.984580498866213
[ 0.00251718+0.00308753j  0.00131765+0.00519702j -0.00272323+0.00164546j
  0.08148193-0.j         -0.00272323-0.00164546j  0.00131765-0.00519702j]


C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['freq2'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['mag2'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['freq2'][indexFreq] = filtered2['freq'][tempIndex]
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:19: SettingWithCopyWarning: 
A value is trying to be 

1.127142689358262e-09

In [55]:
file = 'test3.wav'
recordAudio(file)
filtered = soundFFT(file)
freqMagMSE(filtered)

Recording
Finished recording
440320 44100 9.984580498866213
[-0.00157146+0.00143338j  0.00377742+0.00252503j  0.00244992+0.00214624j
  0.08209229-0.j          0.00244992-0.00214624j  0.00377742-0.00252503j]


C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['freq2'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['mag2'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['freq2'][indexFreq] = filtered2['freq'][tempIndex]
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:19: SettingWithCopyWarning: 
A value is trying to be 

9.809339415421354e-10

In [56]:
file = 'test4.wav'
recordAudio(file)
filtered = soundFFT(file)
freqMagMSE(filtered)

Recording
Finished recording
440320 44100 9.984580498866213
[-0.00121353+0.00138722j  0.00399885+0.0020796j   0.00055033+0.00914153j
  0.08813477-0.j          0.00055033-0.00914153j  0.00399885-0.0020796j ]


C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['freq2'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['mag2'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['freq2'][indexFreq] = filtered2['freq'][tempIndex]
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:19: SettingWithCopyWarning: 
A value is trying to be 

9.437711208782598e-10

In [57]:
file = 'alarmtest1.wav'
recordAudio(file)
filtered = soundFFT(file)
freqMagMSE(filtered)

Recording
Finished recording
440320 44100 9.984580498866213
[ 0.00041623+0.00120335j -0.00490271+0.00204744j -0.01012326+0.00429156j
  0.09536743-0.j         -0.01012326-0.00429156j -0.00490271-0.00204744j]


C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['freq2'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  merged_df['mag2'] = pd.Series()
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['freq2'][indexFreq] = filtered2['freq'][tempIndex]
C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\621228529.py:19: SettingWithCopyWarning: 
A value is trying to be 

1.0985598243153065e-09

In [58]:
# file = 'alarm_mono_single_pi.wav'
# recordAudio(file)
filtered = soundFFT('stream.wav')
freqMagMSE(filtered)

C:\Users\S-_-z\AppData\Local\Temp\ipykernel_19344\722879639.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, sampling_rate = librosa.load(filename, sr = None, mono = True, offset = 0, duration = None)
c:\Users\S-_-z\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'stream.wav'